<a href="https://colab.research.google.com/github/abhi-ar17/TSR/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# Image Classification

# Import libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense

# Initalize CNN
classifier = Sequential()

# Add 2 convolution layers
classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

# Add pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add 2 more convolution layers
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

# Add max pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add 2 more convolution layers
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

# Add max pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add global average pooling layer
classifier.add(GlobalAveragePooling2D())

# Add full connection
classifier.add(Dense(units=15, activation='softmax'))

# Compiling the ANN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit CNN to images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory('fdtju/datasets',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'sw/valid',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

classifier.fit_generator(
        train_set,
        steps_per_epoch=100,
        epochs=5,
        validation_data=test_set,
        validation_steps=40)

classifier.save('model_categorical_complex.h5')

# Test accuracy of classifier
def test_accuracy(classifier, test_set, steps):
    num_correct = 0
    num_guesses = 0
    for i in range(steps):
        a = test_set.next()
        guesses = classifier.predict(a[0])
        correct = a[1]
        for index in range(len(guesses)):
            num_guesses += 1
            if round(guesses[index][0]) == correct[index]:
                num_correct += 1
    return num_correct, num_guesses
        

Found 11884 images belonging to 15 classes.
Found 4308 images belonging to 15 classes.
Epoch 1/5
100/100 [==============================] - 18s 175ms/step - loss: 2.4924 - acc: 0.1366 - val_loss: 1.8087 - val_acc: 0.3430
Epoch 2/5
100/100 [==============================] - 16s 164ms/step - loss: 1.7287 - acc: 0.4109 - val_loss: 1.3319 - val_acc: 0.5898
Epoch 3/5
100/100 [==============================] - 15s 150ms/step - loss: 1.3643 - acc: 0.5456 - val_loss: 1.1778 - val_acc: 0.5516
Epoch 4/5
100/100 [==============================] - 15s 147ms/step - loss: 0.9764 - acc: 0.6759 - val_loss: 0.7029 - val_acc: 0.7666
Epoch 5/5
100/100 [==============================] - 16s 157ms/step - loss: 0.5403 - acc: 0.8319 - val_loss: 0.3347 - val_acc: 0.9133


In [80]:
import cv2
import tensorflow as tf

DATADIR = "fdtju/datasets"

CATEGORIES =["70","50","Cattle","Narrow","No_left","No_Parking","No_right","No_stop","Pedestrian","school","Stop","Two" ,"Hump", "Men","Noentry"]

def prepare(filepath):
	IMG_SIZE=64
	img_array = cv2.imread(filepath)
	new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
	return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)

model=tf.keras.models.load_model("model_categorical_complex.model")

prediction = model.predict([prepare('bcd.jpg')])

print(prediction)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
